In [1]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [11]:
api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=200)

wiki = WikipediaQueryRun(api_wrapper=api_wrapper)


In [12]:
wiki.name

'wikipedia'

In [6]:
from dotenv import load_dotenv
import os
load_dotenv()

API_KEY = os.getenv("GOOGLE_API_KEY")

In [7]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import GooglePalmEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = WebBaseLoader("https://docs.smith.langchain.com/")
docs = loader.load()
documents = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200).split_documents(docs)

vectordb = FAISS.from_documents(documents, GooglePalmEmbeddings(google_api_key=API_KEY))

retriever = vectordb.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'GooglePalmEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x00000211F90BC3A0>)

In [8]:
from langchain.tools.retriever import create_retriever_tool

retrieval_tool = create_retriever_tool(retriever, "langsmith_search",
                      "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!" )


retrieval_tool.name

'langsmith_search'

In [10]:
#arxiv tool
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun

arxiv_wrapper = ArxivAPIWrapper(top_k_results=1, doc_content_chars_max= 200)
arxiv = ArxivQueryRun(api_wrapper=api_wrapper)
arxiv.name

'arxiv'

In [13]:
tools = [wiki, arxiv, retrieval_tool]
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'C:\\Users\\Sheela Sai kumar\\AppData\\Roaming\\Python\\Python310\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=200)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=200)),
 Tool(name='langsmith_search', description='Search for information about LangSmith. For any questions about LangSmith, you must use this tool!', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x00000211DFAA2050>, retriever=VectorStoreRetriever(tags=['FAISS', 'GooglePalmEmbeddings'], vec

In [16]:
## Agents
from langchain.agents import create_openai_tools_agent

agent = create_openai_tools_agent()

In [22]:
from langchain_community.llms.google_palm import GooglePalm
llm = GooglePalm(google_api_key= API_KEY,temperature=0.1)
llm


GooglePalm(client=<module 'google.generativeai' from 'C:\\Users\\Sheela Sai kumar\\AppData\\Roaming\\Python\\Python310\\site-packages\\google\\generativeai\\__init__.py'>, google_api_key=SecretStr('**********'), temperature=0.1)

In [31]:
# Try ollama
from langchain_community.llms import Ollama
llm2 = Ollama(model="llama3.1")
llm2

Ollama(model='llama3.1')

In [55]:
# !pip install transformers

In [54]:
# from langchain_community.llms import HuggingFaceEndpoint
# from langchain_community.chat_models.huggingface import ChatHuggingFace

# llm = HuggingFaceEndpoint(repo_id="HuggingFaceH4/zephyr-7b-beta")

# chat_model = ChatHuggingFace(llm=llm)


In [32]:
from langchain import hub
from langchain.tools.render import render_text_description

prompt = hub.pull("hwchase17/react-json")
print(prompt.messages)
prompt = prompt.partial(
    tools=render_text_description(tools),
    tool_names=", ".join([t.name for t in tools]),
)

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['tool_names', 'tools'], template='Answer the following questions as best you can. You have access to the following tools:\n\n{tools}\n\nThe way you use the tools is by specifying a json blob.\nSpecifically, this json should have a `action` key (with the name of the tool to use) and a `action_input` key (with the input to the tool going here).\n\nThe only values that should be in the "action" field are: {tool_names}\n\nThe $JSON_BLOB should only contain a SINGLE action, do NOT return a list of multiple actions. Here is an example of a valid $JSON_BLOB:\n\n```\n{{\n  "action": $TOOL_NAME,\n  "action_input": $INPUT\n}}\n```\n\nALWAYS use the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction:\n```\n$JSON_BLOB\n```\nObservation: the result of the action\n... (this Thought/Action/Observation can repeat N times)\nThought: I now know the final answer\nF

In [53]:
# from langchain.agents import Tool, AgentExecutor, LLMSingleActionAgent, AgentOutputParser
# from langchain.prompts import StringPromptTemplate
# from langchain.chat_models import AzureChatOpenAI
# from langchain import LLMChain
# from langchain.utilities import BingSearchAPIWrapper
# from typing import List, Union
# from langchain.schema import AgentAction, AgentFinish
# import re
# class CustomOutputParser(AgentOutputParser):
    
#     def parse(self, llm_output: str) -> Union[AgentAction, AgentFinish]:
#         # Check if agent should finish
#         if "Final Answer:" in llm_output:
#             return AgentFinish(
                
#                 return_values={"output": llm_output.split("Final Answer:")[-1].strip()},
#                 log=llm_output)
            
#         # Parse out the action and action input
#         regex = r"Action\s*\d*\s*:(.*?)\nAction\s*\d*\s*Input\s*\d*\s*:[\s]*(.*)"
#         match = re.search(regex, llm_output, re.DOTALL)
#         if not match:
#             raise ValueError(f"Could not parse LLM output: `{llm_output}`")
#         action = match.group(1).strip()
#         action_input = match.group(2)
#         # Return the action and action input
#         return AgentAction(tool=action, tool_input=action_input.strip(" ").strip('"'), log=llm_output)

# output_parser = CustomOutputParser()

In [48]:
# simple output parser
from langchain.agents import Tool, AgentExecutor, LLMSingleActionAgent, AgentOutputParser
from langchain.schema import AgentAction, AgentFinish
from typing import Union

class SimpleOutputParser(AgentOutputParser):
    def parse(self, llm_output: str) -> Union[AgentAction, AgentFinish]:
        return AgentFinish(return_values={"output": llm_output.strip()}, log=llm_output)

In [49]:
# Now create an open source agent
from langchain.agents.format_scratchpad import format_log_to_str
from langchain.agents.output_parsers import (
    ReActJsonSingleInputOutputParser,
)

# Define the output parser
output_parser = SimpleOutputParser()

# Define the agent
chat_model_with_stop = llm2.bind(stop=["\nObservation"])
agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_log_to_str(x["intermediate_steps"]),
    }
    | prompt
    | chat_model_with_stop
    | output_parser
)

In [50]:
agent

{
  input: RunnableLambda(...),
  agent_scratchpad: RunnableLambda(...)
}
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], partial_variables={'tools': "wikipedia - A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.\narxiv - A wrapper around Arxiv.org Useful for when you need to answer questions about Physics, Mathematics, Computer Science, Quantitative Biology, Quantitative Finance, Statistics, Electrical Engineering, and Economics from scientific articles on arxiv.org. Input should be a search query.\nlangsmith_search(query: 'str', *, retriever: 'BaseRetriever' = VectorStoreRetriever(tags=['FAISS', 'GooglePalmEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x00000211F90BC3A0>), document_prompt: 'BasePromptTemplate' = PromptTemplate(input_variables=['page_content'], template='{page_content}'), do

In [51]:
# Agent Executor
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=agent, tools = tools, verbose= True)
agent_executor

AgentExecutor(verbose=True, agent=RunnableAgent(runnable={
  input: RunnableLambda(...),
  agent_scratchpad: RunnableLambda(...)
}
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], partial_variables={'tools': "wikipedia - A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.\narxiv - A wrapper around Arxiv.org Useful for when you need to answer questions about Physics, Mathematics, Computer Science, Quantitative Biology, Quantitative Finance, Statistics, Electrical Engineering, and Economics from scientific articles on arxiv.org. Input should be a search query.\nlangsmith_search(query: 'str', *, retriever: 'BaseRetriever' = VectorStoreRetriever(tags=['FAISS', 'GooglePalmEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x00000211F90BC3A0>), document_prompt: 'BasePromptTemplate' = PromptTemplate(input_v

In [52]:
agent_executor.invoke({"input":"Tell me anything about Langsmith"})



> Entering new AgentExecutor chain...
Question: Tell me anything about Langsmith
Thought: Since this is a question about LangSmith, I should try to find information specifically about it using the langsmith_search tool.
Action:
```
{
  "action": "langsmith_search",
  "action_input": "Langsmith"
}
``

> Finished chain.


{'input': 'Tell me anything about Langsmith',
 'output': 'Question: Tell me anything about Langsmith\nThought: Since this is a question about LangSmith, I should try to find information specifically about it using the langsmith_search tool.\nAction:\n```\n{\n  "action": "langsmith_search",\n  "action_input": "Langsmith"\n}\n``'}